# Datium Data Science Test

### Import dependencies

In [21]:
import os

# suppress console output
os.environ['MLFLOW_LOGGING_ENABLED'] = 'false'

import warnings

# ignore all warnings
warnings.filterwarnings('ignore')

import optuna
import mlflow
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from skimpy import skim

### Load dataset

In [22]:
df_train = pd.read_csv('data/DatiumTrain.rpt', delimiter = '\t', low_memory=False)
df_test = pd.read_csv('data/DatiumTest.rpt', delimiter = '\t', low_memory=False)

### Exploratory Data Analyis

In [23]:
# Summarize training dataframe
skim(df_train)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 50704  │ │ float64     │ 65    │                                                          │
│ │ Number of columns │ 130    │ │ string      │ 59    │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 6     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name             ┃ NA     ┃ NA %    ┃ mean   ┃ sd     ┃ p0    ┃ p25     ┃ p75    ┃ p100    ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩  │
│ │ YearGroup               │      0 │       0 │   2000 │    5.3 │  2000 │    2000 │   2000 │    2000 │    ▃█▇ │  │
│ │ MonthGroup              │      0 │       0 │    1.5 │      3 │     0 │       0 │      1 │      12 │ █   ▁  │  │
│ │ SequenceNum             │      0 │       0 │    9.6 │     11 │     0 │       1 │     15 │     120 │   █▁   │  │
│ │ GearNum                 │      1 │   0.002 │    4.6 │   0.96 │     1 │       4 │      5 │       9 │    ██  │  │
│ │ DoorNum                 │      0 │       0 │    4.1 │   0.87 │     2 │       4 │      5 │       5 │ ▂   █▅ │  │
│ │ EngineSize              │      0 │       0 │   3000 │    930 │   660 │    2400 │   3800 │    7300 │  ▂██▄  │  │
│ │ Cylinders               │      0 │       0 │    5.1 │    1.1 │     2 │       4 │      6 │      12 │   █▇▁  │  │
│ │ FuelCapacity            │    810 │     1.6 │     72 │     22 │    32 │      63 │     75 │     180 │   ▂█   │  │
│ │ GrossCombinationMAss    │  26000 │      52 │   4200 │   1100 │  1400 │    3300 │   4900 │    9100 │ ▁█▇▃▁  │  │
│ │ GrossVehicleMass        │  20000 │      40 │   2400 │    510 │   970 │    2000 │   2800 │    5700 │  ▂█▆   │  │
│ │ WheelBase               │    140 │    0.28 │   2800 │    200 │     2 │    2700 │   2900 │    4300 │    █▄  │  │
│ │ Height                  │    720 │     1.4 │   1600 │    180 │  1100 │    1400 │   1700 │    2900 │   █▃   │  │
│ │ Length                  │    660 │     1.3 │   4800 │    320 │  2500 │    4600 │   4900 │    7300 │   ▁█▃  │  │
│ │ Width                   │    660 │     1.3 │   1800 │     79 │  1400 │    1800 │   1900 │    2400 │   ▂█▃  │  │
│ │ KerbWeight              │   1500 │       3 │   1600 │    280 │   640 │    1400 │   1700 │    3200 │  ▁▅█▁  │  │
│ │ TareMass                │  11000 │      21 │   1600 │    280 │   720 │    1400 │   1700 │    2900 │  ▁▄█▂  │  │
│ │ PayLoad                 │  25000 │      49 │    780 │    310 │   260 │     520 │   1000 │    2700 │  █▅▃   │  │
│ │ Power                   │    110 │    0.22 │    140 │     42 │    32 │     100 │    160 │     450 │  ▃█▃   │  │
│ │ PowerRPMFrom            │  50000 │     100 │   3900 │    800 │  3000 │    3200 │   4500 │    6000 │ █ ▂▂▃  │  │
│ │ PowerRPMTo              │    140 │    0.27 │   5200 │    880 │  2600 │    4900 │   6000 │    8500 │ ▁▃██▁  │  │
│ │ Torque                  │    270 │    0.54 │    290 │     89 │    67 │     220 │    350 │     800 │  ▄██▁  │  │
│ │ TorqueRPMFrom           │  46000 │      91 │   1900 

In [24]:
# Drop columns missing above 80% data
thresh_na = len(df_train)*0.8

df_train = df_train.dropna(thresh=thresh_na, axis=1)
skim(df_train)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 50704  │ │ string      │ 41    │                                                          │
│ │ Number of columns │ 75     │ │ float64     │ 28    │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 6     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name         ┃ NA    ┃ NA %     ┃ mean    ┃ sd     ┃ p0    ┃ p25     ┃ p75     ┃ p100      ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩  │
│ │ YearGroup           │     0 │        0 │    2000 │    5.3 │  2000 │    2000 │    2000 │      2000 │    ▃█▇ │  │
│ │ MonthGroup          │     0 │        0 │     1.5 │      3 │     0 │       0 │       1 │        12 │ █   ▁  │  │
│ │ SequenceNum         │     0 │        0 │     9.6 │     11 │     0 │       1 │      15 │       120 │   █▁   │  │
│ │ GearNum             │     1 │    0.002 │     4.6 │   0.96 │     1 │       4 │       5 │         9 │    ██  │  │
│ │ DoorNum             │     0 │        0 │     4.1 │   0.87 │     2 │       4 │       5 │         5 │ ▂   █▅ │  │
│ │ EngineSize          │     0 │        0 │    3000 │    930 │   660 │    2400 │    3800 │      7300 │  ▂██▄  │  │
│ │ Cylinders           │     0 │        0 │     5.1 │    1.1 │     2 │       4 │       6 │        12 │   █▇▁  │  │
│ │ FuelCapacity        │   810 │      1.6 │      72 │     22 │    32 │      63 │      75 │       180 │   ▂█   │  │
│ │ WheelBase           │   140 │     0.28 │    2800 │    200 │     2 │    2700 │    2900 │      4300 │    █▄  │  │
│ │ Height              │   720 │      1.4 │    1600 │    180 │  1100 │    1400 │    1700 │      2900 │   █▃   │  │
│ │ Length              │   660 │      1.3 │    4800 │    320 │  2500 │    4600 │    4900 │      7300 │   ▁█▃  │  │
│ │ Width               │   660 │      1.3 │    1800 │     79 │  1400 │    1800 │    1900 │      2400 │   ▂█▃  │  │
│ │ KerbWeight          │  1500 │        3 │    1600 │    280 │   640 │    1400 │    1700 │      3200 │  ▁▅█▁  │  │
│ │ Power               │   110 │     0.22 │     140 │     42 │    32 │     100 │     160 │       450 │  ▃█▃   │  │
│ │ PowerRPMTo          │   140 │     0.27 │    5200 │    880 │  2600 │    4900 │    6000 │      8500 │ ▁▃██▁  │  │
│ │ Torque              │   270 │     0.54 │     290 │     89 │    67 │     220 │     350 │       800 │  ▄██▁  │  │
│ │ TorqueRPMTo         │   310 │     0.61 │    3500 │    850 │  1200 │    2800 │    4100 │      7500 │  ▂▇█▄  │  │
│ │ SeatCapacity        │    37 │    0.073 │     4.9 │    1.3 │     2 │       5 │       5 │        15 │  ▁█▁   │  │
│ │ ValvesCylinder      │   510 │        1 │     3.5 │   0.85 │     2 │       4 │       4 │         5 │ ▂   █  │  │
│ │ TowingBrakes        │  2900 │      5.8 │    1800 │    630 │   200 │    1300 │    2200 │      4500 │ ▁██▂▁  │  │
│ │ TowingNoBrakes      │  5200 │       10 │     630 │    140 │   100 │     500 │     750 │      2500 │   ▁█   │  │
│ │ WarrantyYears       │  2600 │      5.2 │     3.1 │  

In [25]:
# Drop rows with no labels
df_train = df_train.dropna(subset=["Sold_Amount"])
skim(df_train)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 50700  │ │ string      │ 41    │                                                          │
│ │ Number of columns │ 75     │ │ float64     │ 28    │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 6     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name         ┃ NA     ┃ NA %    ┃ mean    ┃ sd     ┃ p0    ┃ p25     ┃ p75     ┃ p100      ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩  │
│ │ YearGroup           │      0 │       0 │    2000 │    5.3 │  2000 │    2000 │    2000 │      2000 │    ▃█▇ │  │
│ │ MonthGroup          │      0 │       0 │     1.5 │      3 │     0 │       0 │       1 │        12 │ █   ▁  │  │
│ │ SequenceNum         │      0 │       0 │     9.6 │     11 │     0 │       1 │      15 │       120 │   █▁   │  │
│ │ GearNum             │      0 │       0 │     4.6 │   0.96 │     1 │       4 │       5 │         9 │    ██  │  │
│ │ DoorNum             │      0 │       0 │     4.1 │   0.87 │     2 │       4 │       5 │         5 │ ▂   █▅ │  │
│ │ EngineSize          │      0 │       0 │    3000 │    930 │   660 │    2400 │    3800 │      7300 │  ▂██▄  │  │
│ │ Cylinders           │      0 │       0 │     5.1 │    1.1 │     2 │       4 │       6 │        12 │   █▇▁  │  │
│ │ FuelCapacity        │    810 │     1.6 │      72 │     22 │    32 │      63 │      75 │       180 │   ▂█   │  │
│ │ WheelBase           │    140 │    0.28 │    2800 │    200 │  1400 │    2700 │    2900 │      4300 │    █▄  │  │
│ │ Height              │    720 │     1.4 │    1600 │    180 │  1100 │    1400 │    1700 │      2900 │   █▃   │  │
│ │ Length              │    660 │     1.3 │    4800 │    320 │  2500 │    4600 │    4900 │      7300 │   ▁█▃  │  │
│ │ Width               │    660 │     1.3 │    1800 │     79 │  1400 │    1800 │    1900 │      2400 │   ▂█▃  │  │
│ │ KerbWeight          │   1500 │       3 │    1600 │    280 │   640 │    1400 │    1700 │      3200 │  ▁▅█▁  │  │
│ │ Power               │    110 │    0.22 │     140 │     42 │    32 │     100 │     160 │       450 │  ▃█▃   │  │
│ │ PowerRPMTo          │    140 │    0.27 │    5200 │    880 │  2600 │    4900 │    6000 │      8500 │ ▁▃██▁  │  │
│ │ Torque              │    270 │    0.54 │     290 │     89 │    67 │     220 │     350 │       800 │  ▄██▁  │  │
│ │ TorqueRPMTo         │    310 │    0.61 │    3500 │    850 │  1200 │    2800 │    4100 │      7500 │  ▂▇█▄  │  │
│ │ SeatCapacity        │     36 │   0.071 │     4.9 │    1.3 │     2 │       5 │       5 │        15 │  ▁█▁   │  │
│ │ ValvesCylinder      │    510 │       1 │     3.5 │   0.85 │     2 │       4 │       4 │         5 │ ▂   █  │  │
│ │ TowingBrakes        │   2900 │     5.8 │    1800 │    630 │   200 │    1300 │    2200 │      4500 │ ▁██▂▁  │  │
│ │ TowingNoBrakes      │   5200 │      10 │     630 │    140 │   100 │     500 │     750 │      2500 │   ▁█   │  │
│ │ WarrantyYears       │   2600 │     5.2 │     3.1 │  

In [26]:
# Define features and target
columns_to_drop = [
    "AvgWholesale",
    "AvgRetail",
    "GoodWholesale",
    "GoodRetail",
    "TradeMin",
    "TradeMax",
    "PrivateMax",
]
for column in columns_to_drop:
    if column in df_train.columns:
        df_train = df_train.drop(column, axis=1)

X_train = df_train.drop(["Sold_Amount"], axis=1)
y_train = df_train["Sold_Amount"]

X_test = df_test.drop(["Sold_Amount"], axis=1)
y_test = df_test["Sold_Amount"]

features = X_train.columns.to_list()

### Data Preprocessing

In [27]:
cat_cols = X_train.select_dtypes(exclude="number").columns
num_cols = X_train.select_dtypes(include="number").columns

In [28]:
# Categorical features
categorical_pipeline = Pipeline(
    steps=[
        ("SimpleImputer", SimpleImputer(strategy="most_frequent")),
        ("OneHotEncoder", OneHotEncoder(handle_unknown='ignore'))
    ]
)

# Numerical features
numeric_pipeline = Pipeline(
    steps=[("SimpleImputer", SimpleImputer(strategy="mean")), 
        ("StandardScaler", StandardScaler())]
)

full_processor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, num_cols),
        ("categorical", categorical_pipeline, cat_cols),
    ]
)

# Fit and transform the encoder on the training data
X_train_encoded = full_processor.fit_transform(X_train, y_train)

# Transform the encoder on the testing data
X_test_encoded = full_processor.transform(X_test)

### Modelling

In [29]:
class LGBMRegressorOptuna:
    def __init__(self, num_boost_round=100, n_trials=100):
        self.num_boost_round = num_boost_round
        self.n_trials = n_trials
        self.best_params = None
        self.run_name = "LGBMRegressorOptuna"
    
    def fit(self, X_train, y_train, X_test, y_test):
        # Start an MLflow run
        with mlflow.start_run(run_name=self.run_name):
            # Define the objective function for Optuna
            def objective(trial):
                # Define the hyperparameters to tune
                params = {
                    'objective': 'regression',
                    'metric': 'rmse',
                    'num_leaves': trial.suggest_int('num_leaves', 10, 100),
                    'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
                    'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
                    'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
                    'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
                    'min_child_samples': trial.suggest_int('min_child_samples', 1, 50),
                    'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.001, 10.0),
                    'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.001, 10.0),
                }

                # Convert the training data into LightGBM format
                train_data = lgb.Dataset(X_train, label=y_train)

                # Train the LightGBM model
                model = lgb.train(params, train_data, num_boost_round=self.num_boost_round)

                # Use the trained model to make predictions on the test data
                y_pred = model.predict(X_test)

                # Calculate the root mean squared error (RMSE) of the predictions
                rmse = mean_squared_error(y_test, y_pred, squared=False)

                # Return the RMSE as the objective value to minimize
                return rmse

            # Split the dataset into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

            # Create an Optuna study
            study = optuna.create_study(direction='minimize')

            # Optimize the study using the objective function
            study.optimize(objective, n_trials=self.n_trials)

            # Store the best hyperparameters
            self.best_params = study.best_trial.params

            # Train a final LightGBM model using the best hyperparameters
            self.model = lgb.train(self.best_params, lgb.Dataset(X_train, label=y_train), num_boost_round=self.num_boost_round)

            y_pred_test = self.model.predict(X_test)
            self.mse_test = mean_squared_error(y_test, y_pred_test)
            self.rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
            self.r2_test = r2_score(y_test, y_pred_test)

            # Log the best hyperparameters as MLflow tags
            mlflow.set_tags(self.best_params)

            # Log the RMSE and hyperparameters as MLflow parameters
            mlflow.log_params(self.best_params)
            mlflow.log_metric("mse", self.mse_test)
            mlflow.log_metric("rmse", self.rmse_test)
            mlflow.log_metric("r2", self.r2_test)

            mlflow.sklearn.log_model(self.model, "model")

            # Print the best hyperparameters and the best RMSE achieved
            print('Best hyperparameters:', self.best_params)
            print('Best MSE  :', self.mse_test)
            print('Best RMSE :', self.rmse_test)
            print('Best R2   :', self.r2_test)

    def predict(self, X):
        return self.model.predict(X)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)


In [30]:
# Initialize the LGBMRegressorOptuna class
lgbm = LGBMRegressorOptuna(num_boost_round=100, n_trials=100)

# Fit the model to the data
lgbm.fit(X_train_encoded, y_train, X_test_encoded, y_test)

# Make predictions on new data
y_pred = lgbm.predict(X_test_encoded)

# Compute the R^2 score on the test data
r2 = lgbm.score(X_test_encoded, y_test)

[I 2023-03-16 02:49:14,993] A new study created in memory with name: no-name-ba545a62-94f7-4e14-a21c-9716acf05e97


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022156 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7931
[LightGBM] [Info] Number of data points in the train set: 40560, number of used features: 2306
[LightGBM] [Info] Start training from score 16423.053517


[I 2023-03-16 02:49:15,628] Trial 0 finished with value: 8009.657268735604 and parameters: {'num_leaves': 14, 'learning_rate': 0.003437922316116085, 'feature_fraction': 0.5583190891405722, 'bagging_fraction': 0.233576154900408, 'bagging_freq': 4, 'min_child_samples': 30, 'reg_alpha': 0.004741113188703143, 'reg_lambda': 0.010413465868789358}. Best is trial 0 with value: 8009.657268735604.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029717 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15157
[LightGBM] [Info] Number of data points in the train set: 40560, number of used features: 5919
[LightGBM] [Info] Start training from score 16423.053517


[I 2023-03-16 02:49:18,123] Trial 1 finished with value: 3066.0116080686685 and parameters: {'num_leaves': 71, 'learning_rate': 0.04143378561406797, 'feature_fraction': 0.8899959573539186, 'bagging_fraction': 0.5099037700262828, 'bagging_freq': 3, 'min_child_samples': 10, 'reg_alpha': 2.5407627178981276, 'reg_lambda': 0.05088075743820339}. Best is trial 1 with value: 3066.0116080686685.


...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028223 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17169
[LightGBM] [Info] Number of data points in the train set: 40560, number of used features: 6925
[LightGBM] [Info] Start training from score 16423.053517


[I 2023-03-16 02:53:43,610] Trial 98 finished with value: 2860.9889693980817 and parameters: {'num_leaves': 96, 'learning_rate': 0.09131326877892325, 'feature_fraction': 0.8020665552009496, 'bagging_fraction': 0.23756878737310466, 'bagging_freq': 1, 'min_child_samples': 8, 'reg_alpha': 6.891664073755378, 'reg_lambda': 1.2594177438259198}. Best is trial 83 with value: 2805.9527251651957.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024712 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16115
[LightGBM] [Info] Number of data points in the train set: 40560, number of used features: 6398
[LightGBM] [Info] Start training from score 16423.053517


[I 2023-03-16 02:53:45,890] Trial 99 finished with value: 2886.2040293946625 and parameters: {'num_leaves': 92, 'learning_rate': 0.09108103136847853, 'feature_fraction': 0.7742344506624493, 'bagging_fraction': 0.20557244361613375, 'bagging_freq': 1, 'min_child_samples': 9, 'reg_alpha': 8.271182548391527, 'reg_lambda': 1.8576220744674532}. Best is trial 83 with value: 2805.9527251651957.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045592 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15157
[LightGBM] [Info] Number of data points in the train set: 40560, number of used features: 5919
[LightGBM] [Info] Start training from score 16423.053517
Best hyperparameters: {'num_leaves': 86, 'learning_rate': 0.09018960900118386, 'feature_fraction': 0.9128188556941866, 'bagging_fraction': 0.3142492548905205, 'bagging_freq': 1, 'min_child_samples': 10, 'reg_alpha': 2.986719312955511, 'reg_lambda': 0.5067009639186371}
Best MSE  : 7873370.69586199
Best RMSE : 2805.9527251651957
Best R2   : 0.9156389250279962


### Model Performance

| Model           | Metrics         | Score           |
| --------------- | --------------- | --------------- |
| LightGBM        | RMSE            | 2,806           |
|                 | R2              | 0.916           |


### Discussion

- In this experiment, I started with exploring simpler models e.g. Linear Regression, Ridge and Lasso, paired with different permutations of feature encoders e.g. One Hot Encoder, Ordinal Encoder and Standard Scaler
- It is found that **Ridge model** with **One Hot Encoder + Standard Scaler** works best: 
    - R2: 0.769 
    - RMSE: 4,945
- Next, I explored using another regression model - **LightGBM Regression**, paired with One Hot Encoder + Standard Scaler preprocessing steps
- After auto-tuning using Optuna, the best performance is:
    - R2: 0.916
    - RMSE: 2,806

### Potential improvements
- Using Feature Importance to decide which features should be included and excluded
- Train on more data